In [2]:
#решить две системы (одну с “точной” правой частью, вторую с изменённой), 
#найти число обусловленности исходной матрицы, посчитать фактическую погрешность и 
#оценку погрешности (задание - 10.4, варианты - 10.5 по методичке, стр.90). 
#Затем реализовать три метода: Гаусса с выбором главного элемента (по желанию) 
#    для решения СЛАУ, метод Жордана для нахождения обратной матрицы и метод LU-разложения
 #   для нахождения определителя матрицы (варианты - 11.6 по методичке, стр.94).
    
    
#Везде взят вариант 1

In [3]:
import pandas as pd
import numpy as np

In [4]:
matrixA = np.array([[-400.60, 199.80], [1198.80, -600.40]])
#Обратная к А матрица
matrixA1 = np.linalg.inv(matrixA)
matrixB = np.array([200.0, -600.0])
matrixB1 = np.array([199.0, -601.0])

In [5]:
#Решение системы с точной правой частью
answer = np.linalg.solve(matrixA, matrixB)
print(answer)

[-0.2  0.6]


In [6]:
#Решение системы с измененной правой частью
answer1 = np.linalg.solve(matrixA, matrixB1)
print(answer1)

[0.6002 2.1994]


In [7]:
#Число обусловленности исходной матрицы (норма inf )
condA = np.linalg.cond(matrixA, np.inf)

print(condA)

2877.6404800000278


In [8]:
#Фактическая относительная погрешность
temp = answer1 - answer
relative_error = (np.linalg.norm(temp, np.inf) / np.linalg.norm(answer, np.inf))
print(relative_error)

2.665666666666609


In [9]:
#Оценка погрешности(relative_error <=)
tempB = matrixB1 - matrixB
deltaB = (np.linalg.norm(tempB, np.inf) / np.linalg.norm(matrixB, np.inf))
error_estimation = condA * deltaB
print(error_estimation)

4.796067466666713


In [10]:
#Метод Гаусса с выбором главного элемента
def Metod_Gaussa(arr, brr):
    for k in range(arr.shape[0] - 1):
        # Поиск строки с максимальным элементом
        max_elem = 0
        str = 0
        for i in range (k, arr.shape[0]):
            if abs(arr[i,k]) > abs(max_elem):
                max_elem = arr[i,k]
                str = i
        # Меняем местами строки квадратной матрицы
        change = np.repeat(arr[k], 1)
        arr[k], arr[str] = arr[str], change
        # Меняем местами элементы вектора-столбца
        change = np.repeat(brr[k], 1)
        brr[k], brr[str] = brr[str], change
        # Делим полученную строку на max_elem
        arr[k] = arr[k] / max_elem
        brr[k] = brr[k] / max_elem
        # Домножаем строку на коэффициенты и вычитаем ее из остальных строк
        for i in range (k + 1, arr.shape[0]):
            factor = arr[i,k]
            arr[i] = arr[i] - arr[k] * factor
            brr[i] = brr[i] - brr[k] * factor

    # Находим аргументы уравнений
    arg = [brr[brr.shape[0] - 1] / (arr[arr.shape[0] - 1, arr.shape[0] - 1])]
    for i in range(arr.shape[0] - 2, -1, -1):
        n = brr[i]
        for j in range(len(arg)):
            n = n - arg[j] * arr[i, arr.shape[0] - 1 - j]
        arg.append(n)

    # Переворачиваем значения в списке
    otv = []
    print("Answer:")
    for i in reversed(arg): otv.append(i)
    print("\n".join("X{0} =\t{1:10.15f}".format(i + 1, x) for i, x in enumerate(otv)))
    return otv

In [11]:
matrixRas = np.array([[3.278164,  1.046583,  -1.378574], [1.046583,  2.975937,   0.934251], [-1.378574,  0.934251,  4.836173]])
matrixResult = np.array([-0.527466, 2.526877, 5.165441])

In [12]:
#вызов функции и вывод на экран
print(Metod_Gaussa(matrixRas, matrixResult))

Answer:
X1 =	0.100000193135086
X2 =	0.499999630025553
X3 =	1.000000105848158
[0.10000019313508599, 0.4999996300255529, 1.0000001058481585]


In [13]:
matrixRas1 = np.array([[3.278164,  1.046583,  -1.378574], [1.046583,  2.975937,   0.934251], [-1.378574,  0.934251,  4.836173]])
matrixResult1 = np.array([-0.527466, 2.526877, 5.165441])

In [14]:
# Метод Жордана для нахождения обратной матрицы

# Перемена местами двух строк системы
def SwapRows(A, B, row1, row2):
    A[row1], A[row2] = A[row2], A[row1]
    B[row1], B[row2] = B[row2], B[row1]

# Деление строки системы на число
def DivideRow(A, B, row, divider):
    A[row] = [a / divider for a in A[row]]
    B[row] /= divider

# Cложение строки системы с другой строкой, умноженной на число
def CombineRows(A, B, row, source_row, weight):
    A[row] = [(a + k * weight) for a, k in zip(A[row], A[source_row])]
    B[row] += B[source_row] * weight

# Решение
def Gauss_Jordan3(A, B):
    column = 0
    while (column < len(B)):
        # Ищем максимальный по модулю элемент в i-м столбце
        current_row = None
        for r in range(column, len(A)):
            if current_row is None or abs(A[r][column]) > abs(A[current_row][column]):
                 current_row = r
        if current_row is None:
            print("решений нет")
            return None
        if current_row != column:
            #Переставляем строку с найденным элементом повыше
            SwapRows(A, B, current_row, column)
        #Нормализуем строку с найденным элементом
        DivideRow(A, B, column, A[column][column])
        #Обрабатываем нижележащие строки
        for r in range(column + 1, len(A)):
            CombineRows(A, B, r, column, -A[r][column])
        column += 1
    #Матрица приведена к треугольному виду, считаем решение
    X = [0 for b in B]
    for i in range(len(B) - 1, -1, -1):
        X[i] = B[i] - sum(x * a for x, a in zip(X[(i + 1):], A[i][(i + 1):]))
    print("Answer:")
    print("\n".join("X{0} =\t{1:10.15f}".format(i + 1, x) for i, x in enumerate(X)))
    return X

In [15]:
Gauss_Jordan3(matrixRas1, matrixResult1)

Answer:
X1 =	0.100000193135086
X2 =	0.499999630025553
X3 =	1.000000105848158


[0.10000019313508599, 0.4999996300255529, 1.0000001058481585]

In [16]:
matrixRas2 = np.array([[3.278164,  1.046583,  -1.378574], [1.046583,  2.975937,   0.934251], [-1.378574,  0.934251,  4.836173]])
matrixResult2 = np.array([-0.527466, 2.526877, 5.165441])

In [17]:
# Разбиваем матрицу коэффициентов на L и U матрицы. Получившиеся треугольные матрицы представляем в виде общей матрицы
def decompose_to_LU(a):

    # Пустая LU-матрица
    lu_matrix = np.matrix(np.zeros([a.shape[0], a.shape[1]]))
    n = a.shape[0]

    for k in range(n):
        for j in range(k, n): #Вычисляем все оставшиеся эл-ты к-й строки
            lu_matrix[k, j] = a[k, j] - lu_matrix[k, :k] * lu_matrix[:k, j]
        for i in range(k + 1, n): #Вычисляем все оставшиеся эл-ты к-го столбца
            lu_matrix[i, k] = (a[i, k] - lu_matrix[i, : k] * lu_matrix[: k, k]) / lu_matrix[k, k]

    return lu_matrix

In [18]:
# Получим матрицу L из матрицы LU
def get_L(m):
    L = m.copy()
    for i in range(L.shape[0]):
            L[i, i] = 1
            L[i, i+1 :] = 0
    return np.matrix(L)

# Получим матрицу U из матрицы LU
def get_U(m):
    U = m.copy()
    for i in range(1, U.shape[0]):
        U[i, :i] = 0
    return U

# Проверка на то, правильно ли разбиваются матрицы
print(matrixRas2)

LU = decompose_to_LU(matrixRas2)
L = get_L(LU)
U = get_U(LU)

print(L * U)

[[ 3.278164  1.046583 -1.378574]
 [ 1.046583  2.975937  0.934251]
 [-1.378574  0.934251  4.836173]]
[[ 3.278164  1.046583 -1.378574]
 [ 1.046583  2.975937  0.934251]
 [-1.378574  0.934251  4.836173]]


In [19]:
# Находим решение метода LU-разложения
def solve_LU(lu_matrix, b):
    y = np.matrix(np.zeros([lu_matrix.shape[0], 1]))
    for i in range(y.shape[0]):
        y[i] = b[i] - lu_matrix[i, :i] * y[:i]

    x = np.matrix(np.zeros([lu_matrix.shape[0], 1]))
    for i in range(1, x.shape[0] + 1):
        x[-i] = (y[-i] - lu_matrix[-i, -i:] * x[-i:] )/ lu_matrix[-i, -i]
    print("Answer:")
    print("X1 = ", x[0][0])
    print("X2 = ", x[1][0])
    print("X3 = ", x[2][0])
    #print("\n".join("X{0} =\t{1:10.15f}".format(i + 1, x_tr[0, k]) for i, k in range(len(x_tr)))
    return x

In [20]:
matrix = decompose_to_LU(matrixRas2)
solve_LU(matrix, matrixResult2)

Answer:
X1 =  [[0.10000019]]
X2 =  [[0.49999963]]
X3 =  [[1.00000011]]


matrix([[0.10000019],
        [0.49999963],
        [1.00000011]])

In [21]:
# Определитель матрицы методом LU разложения
detA = np.linalg.det(L) * np.linalg.det(U)
print(detA)

30.669790066106653
